In [ ]:
# for the LLM

!pip3 install openai

In [ ]:
# only for OCR 

#!pip3 install langdetect
#!pip3 install pytesseract
#!pip3 install pdf2image Pillow

In [26]:
import pandas as pd
import re
from openai import OpenAI

## 1. Initial Text Extraction + cleaning

#### Using PyMuPDF

In [103]:
import fitz  # Import the PyMuPDF library

def extract_text_pymupdf(pdf_path):
    doc = fitz.open(pdf_path)  # Open the PDF file
    text = ""  # Initialize an empty string to store text

    for page in doc:  # Iterate through each page
        text += page.get_text()  # Extract text from the page and append it

    doc.close()  # Close the document
    return text

# here change path to pdf
#pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-2022-DG-Press-HoldinG-B.V.pdf'
pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-Momo-Medical-Holding-B.V.-2022.pdf'
# pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-2022-Informed-IT-Holding-B.V.pdf'
#pdf_path = 'KVK Sample Files 2 - Julia/01016572-docType-sd_jaarrek_art394_lid1-docJaar-2022-docCreatie-2023-12-21-docId-090299cc61e0f819.pdf'
raw_text = extract_text_pymupdf(pdf_path)
text = extract_text_pymupdf(pdf_path)
text_data = extract_text_pymupdf(pdf_path)
print(text)

The accountant's compilation report has been issued
 
 
 
 
 
Consolidated financial statements 2022
Momo Medical Holding B.V.
DELFT
Table of contents
Consolidated balance sheet . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2
Consolidated profit and loss account . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4
Notes to the consolidated financial statements . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5
Balance sheet . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 14
Profit and loss account . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 16
Notes to the financial statements . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 17
Specifications to the financial statements . . . . . . . . . . . . . . . . . . . . . . . . 

In [104]:
# for the first 1000 characters of the text, delete all numeric values that are smaller than 100
# this is to avoid extracting page numbers and other irrelevant data
# is_number returns a number stripped from commas, dots and parentheses and a boolean indicating if the input is a number
# or the raw data if `s` is not a number

MAX_DESCRIPTION_LENGTH = 220
LLM_length=100

def is_number(s: str) -> [str, bool]:
    if s.startswith("(") and s.endswith(")"):
        s = s[1:-1]
    elif s == "-":
        return 0, True
    # NOTE: more cases here
    num = s.replace(",", ".").replace(".", "")
    return num, num.isdigit()

def is_year(s) -> bool: return 2000 <= int(s) <= 2025

def ends_with_percent(s: str) -> bool: return s.endswith("%")

def is_page_number(s: str) -> bool:   return s.strip().startswith("Page") or s.strip().startswith("Pagina") 

# function added to skip over the table of content page numbers
def remove_small_numbers(text, threshold=100):
    # Limit the text to the first 1000 characters for processing
    initial_segment = text
    # Remove numbers smaller than 100
    modified_segment = re.sub(r'\b\d{1,2}\b', '', initial_segment)
    combined_text = modified_segment 
    
   # Remove empty lines
    lines = combined_text.split('\n')
    non_empty_lines = [line for line in lines if line.strip() != '']
    
    # Join the non-empty lines back together
    cleaned_text = '\n'.join(non_empty_lines)
    
    return cleaned_text

def remove_empty_lines(text):
    # Split the text into lines
    lines = text.split('\n')
    
    # Remove empty lines
    non_empty_lines = [line for line in lines if line.strip() != '']
    
    # Join the non-empty lines back together
    cleaned_text = '\n'.join(non_empty_lines)
    
    return cleaned_text

def parse_adjusted_financial_text(text):
    text = remove_small_numbers(text)
    text=remove_empty_lines(text)
     
    # Split the text into lines
    lines = text.strip().split("\n")
    non_empty_lines = [line for line in lines if line.strip() != '']
    
    # Join the non-empty lines back together
    cleaned_text = '\n'.join(non_empty_lines)
    
    # Initialize an empty list to store our parsed data
    parsed_data = []
    
    # this variable tracks if we found any numeric data on the current page
    no_numeric_data_on_page = True
    misc_text_data = []
    
    # Initialize empty strings for current description and values
    current_description = ""
    value_year1 = ""
    
    # Iterate over each line in the text
    for line in lines:
        # Check if line is a description or a value
        maybe_num, is_num = is_number(line)
        
        if is_num and is_year(maybe_num):
            continue
        elif ends_with_percent(line):
            continue
        
        if is_num:
            # If it's a digit, it's a value
            if value_year1 == "":
                # If the first value is not yet set, set it as year1 value
                value_year1 = maybe_num
            else:
                print([current_description.strip().lower(), int(value_year1), int(maybe_num)])
                parsed_data.append([current_description.strip().lower(), int(value_year1), int(maybe_num)])
                # Reset the values for the next set of data
                current_description = ""
                value_year1 = ""
                no_numeric_data_on_page = False
        else:
            # we might find long streches of text unrelated to data labeling
            # in that case we have the following solutions:
            #  * cap descripiton length (x chars)
            #  * try to extract category name using llms
            # If it's not a digit, it's a description
            # Accumulate descriptions until we reach a digit
            if current_description:
                if is_page_number(line):
                    if no_numeric_data_on_page:
                        misc_text_data.append(current_description)
                        current_description = ""
                    # we want to reset the tracker at every new page
                    no_numeric_data_on_page = True
                else: current_description += " " + line.strip()
               
                #skip if the length is more than MAX_DESCRIPTION_LENGTH
                if len(current_description) > MAX_DESCRIPTION_LENGTH:
                    current_description = ""        
            else:
                current_description = line
              
    # Convert the parsed data into a DataFrame
    return pd.DataFrame(parsed_data), misc_text_data
    #return pd.DataFrame(parsed_data, columns=['Description', 'Year 1 EUR', 'Year 2 EUR']), misc_text_data

In [105]:
# Parse the adjusted text data
df_parsed, misc_text_data = parse_adjusted_financial_text(text_data)

# Display the parsed DataFrame
# print(df_parsed)

# df_parsed.to_csv('parsed_data.csv', index=False)


['page  momo medical holding b.v., delft consolidated balance sheet as at  december 2022 after appropriation of results eur eur assets non-current assets property, plant and equipment machinery', 292, 719]
['other tangible assets', 784641, 214361]
['total of property, plant and equipment', 828933, 228080]
['total of non-current assets', 828933, 228080]
['current assets inventories raw materials and consumables', 266885, 896]
['inventories under construction', 450, 64]
['finished and trade goods', 125005, 284]
['total of inventories', 434340, 150244]
['receivables trade receivables', 182558, 229329]
['current receivables from other legal entities and companies with a participating interest in the legal entity or from participating interests of the legal entity taxes and social security charges', 647, 149]
['prepayments and accrued income', 46, 505757]
['total of receivables', 200301, 742285]
['cash and cash equivalents', 219434, 377]
['total of current assets', 854075, 936906]
['total o

In [67]:
# Parse the adjusted text data
df_parsed, misc_text_data = parse_adjusted_financial_text(text_data)

# Display the parsed DataFrame
#display(df_parsed)   

 

In [106]:
import re  # Regular expression module

#TODO delete values of 100, delete ' signs 
def is_description_valid(description):
    # Ensure the description is a string
    if isinstance(description, bytes):
        description = description.decode('utf-8') 

    # Remove all non-alphanumeric characters except spaces, then strip excess whitespace
    clean_description = re.sub(r'[^a-zA-Z0-9\s]', '', description).strip()
    # Check if there's any alphanumeric content left
    return bool(re.search(r'\w', clean_description))

def refine_parsed_data(parsed_data):

    refined_data = []
    for i in range(len(parsed_data)):
        # Handle cases where entries may have variable number of columns
        row = parsed_data.iloc[i] if hasattr(parsed_data, 'iloc') else parsed_data[i]
        description = row[0]
        values = row[1:]

        # Check for valid description
        if is_description_valid(description):
            if refined_data and all(isinstance(v, int) and isinstance(prev_v, int) for v, prev_v in zip(values, refined_data[-1][1:])):
                # Add current line's numbers to the previous line's numbers
                refined_data[-1][1:] = [v + prev_v for v, prev_v in zip(values, refined_data[-1][1:])]
            else:
                refined_data.append([description] + list(values))

    return refined_data


refined_data = refine_parsed_data(df_parsed)
print(refined_data)


[['page  momo medical holding b.v., delft consolidated balance sheet as at  december 2022 after appropriation of results eur eur assets non-current assets property, plant and equipment machinery', 292, 719], ['other tangible assets', 784641, 214361], ['total of property, plant and equipment', 828933, 228080], ['total of non-current assets', 828933, 228080], ['current assets inventories raw materials and consumables', 266885, 896], ['inventories under construction', 450, 64], ['finished and trade goods', 125005, 284], ['total of inventories', 434340, 150244], ['receivables trade receivables', 182558, 229329], ['current receivables from other legal entities and companies with a participating interest in the legal entity or from participating interests of the legal entity taxes and social security charges', 647, 149], ['prepayments and accrued income', 46, 505757], ['total of receivables', 200301, 742285], ['cash and cash equivalents', 219434, 377], ['total of current assets', 854075, 936

### Using OCR

In [9]:
# need to install poppler and tesseract (brew install poppler tesseract)
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import re

class Page:
    def __init__(self, text):
        self.lines = []
        for line in text.strip().split('\n'):
            # TODO: additonal cleaning steps here
            cl = line.strip()
            if cl != "":
                self.lines.append(cl)
            
    def __len__(self):
        acc = 0
        for line in self.lines:
            acc += len(line)
        return acc
    
    def num_lines(self): return len(self.lines)
        
    def apply_to_lines(self, func):
        self.lines = [func(line) for line in self.lines]

class Document:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
        self.pages = self._extract_text_from_pdf()

    def _extract_text_from_pdf(self):
        lang = 'eng+nld' # both English and Dutch
        # convert PDF to a list of images
        pages = []
        
        for page in convert_from_path(self.pdf_path, 500): # here change DPI 
            text = pytesseract.image_to_string(page, lang=lang)
            pages.append(Page(text))

        return pages

    def get_page(self, page_number) -> Page:
        return self.pages[page_number]

    # get min_chars returns the first n pages which characters sum up to at least min_chars
    # used to get the contents table from the begininng of the document
    def get_min_chars(self, min_chars):
        acc = 0
        ret_pages = []
        for page in self.pages:
            acc += len(page)
            ret_pages.append(page)
            if acc >= min_chars:
                return ret_pages
            
    def insert_pages(self, pages, overwrite=False):
        if overwrite: # start overwriting pages from the beginning
            self.pages = pages + self.pages[len(pages):]
        else:
            self.pages += pages

In [10]:
# defining functions for cleaning the output

class DocumentCleaner:
    def __init__(self):
        self.dupa = True

    @staticmethod
    def try_or_false(func, error_type):
        try:
            return func()
        except error_type:
            return False

    @staticmethod
    def is_year(s):
        try:
            num = int(s.replace(',', '').replace('.', ''))
            return 2000 <= num <= 2025
        except ValueError:
            return False
            
        #return DocumentCleaner.try_or_false(lambda: , ValueError)

    @staticmethod
    def delete_year_numbers(line):
        words = line.split()
        return ' '.join(word for word in words if not DocumentCleaner.is_year(word))

    @staticmethod
    def remove_small_numbers(line):
        def should_remove(word):
            try:
                # Check if word is a number (integer or float) and less than 100
                num = float(word)
                return num < 100
            except ValueError:
                return False
        words = line.split()
        return ' '.join(word for word in words if not (len(word) <= 2 or should_remove(word)))

    @staticmethod
    def remove_percentages(line):
        return re.sub(r'\b\d{1,2}\b%', '', line)

    @staticmethod
    def remove_brackets(line):
        return re.sub(r'\((\d+)\)', r'-\1', line) 

    @staticmethod
    def is_page_number(line):
        return line.startswith("Pagina")

    @staticmethod
    def remove_underscores(line):
        return line.replace('_', ' ')

    @staticmethod
    def replace_dash_with_zero(line):
        return line.replace('-', '0').replace(':', '0')

    @staticmethod
    def remove_currency_symbols(line):
        return line.replace('€', '').replace('$', '')
    
    @staticmethod
    def delete_long_words(line):
        words = line.split()
        return ' '.join(word for word in words if len(word) <= 100)

    @staticmethod
    def clean_line(line):
        if not DocumentCleaner.is_page_number(line):
            line = DocumentCleaner.remove_small_numbers(line)
            line = DocumentCleaner.delete_year_numbers(line)
            line = DocumentCleaner.remove_percentages(line)
            line = DocumentCleaner.remove_currency_symbols(line)
            line = DocumentCleaner.remove_underscores(line)
            line = DocumentCleaner.replace_dash_with_zero(line)
            line = DocumentCleaner.remove_brackets(line)
            line = DocumentCleaner.delete_long_words(line)
        return line

    def remove_contents(self, document: Document, table_len=1500):
        clean_table_of_contents = []
        table_of_contents = document.get_min_chars(table_len)
        for page in table_of_contents:
            clean_table_of_contents.append(page.apply_to_lines(DocumentCleaner.remove_small_numbers))
        document.insert_pages(clean_table_of_contents, overwrite=True)

    def clean_page(self, page: Page):
        page.apply_to_lines(DocumentCleaner.clean_line)
        
    def clean_document(self, document: Document):
        for page in document.pages:
            self.clean_page(page)

In [11]:
# here change path to pdf

# pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-2022-DG-Press-HoldinG-B.V.pdf'
pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-Momo-Medical-Holding-B.V.-2022.pdf'
#pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-2022-Informed-IT-Holding-B.V.pdf'
# pdf_path = 'KVK Sample Files 2 - Julia/01042818-docType-sd_jaarrek_art394_lid1-docJaar-2022-docCreatie-2023-12-28-docId-090299cc626872c5.pdf'
# pdf_path = 'KVK Sample Files 2 - Julia/01016572-docType-sd_jaarrek_art394_lid1-docJaar-2022-docCreatie-2023-12-21-docId-090299cc61e0f819.pdf'
#text = extract_text_from_pdf(pdf_path)
#text_data = text
#print(text)
doc = Document(pdf_path)
text=doc
text_description = doc

# for text, join all the lines in the text
text = "\n".join([line for page in doc.pages for line in page.lines])



In [12]:
for page in doc.pages:
    print(page.lines)
    print(page.num_lines())
    print("---------")
    
    
#print page 6
print(doc.get_page(5).lines)
print("---------")    

["The accountant's compilation report has been issued"]
1
---------
['Consolidated financial statements 2022', 'Momo Medical Holding B.V.', 'DELFT']
3
---------
['Table of contents', 'Consolidated balance sheet... .... eee 2', 'Consolidated profit and loss account ................ee ee 4', 'Notes to the consolidated financial statements... neee 5', 'Balance sheet ............ eee 14', 'Profit and loss account... 1... 0.0... cc ee ee eee eee nes 16', 'Notes to the financial statements... .… eee 17', 'Specifications to the financial statements ......... 0.0.0.0... ccc ee es 23']
8
---------
['Page 2', 'Momo Medical Holding B.V., DELFT', 'Consolidated balance sheet as at 31 december 2022', 'After appropriation of results', '2022 2021', 'EUR EUR', 'Assets', 'Non-current assets', 'Property, plant and equipment', 'Machinery 44 292 13,719', 'Other tangible assets 784,641 214,361', 'Total of property, plant and equipment 828,933 228,080', 'Total of non-current assets 828,933 228,080', 'Current

In [13]:
cleaner = DocumentCleaner()
cleaner.clean_document(doc)
num_pages = len(doc.pages)

for i in range(num_pages):
    print(doc.get_page(i).lines)
    print("---------")



["The accountant's compilation report has been issued"]
---------
['Consolidated financial statements', 'Momo Medical Holding B.V.', 'DELFT']
---------
['Table contents', 'Consolidated balance sheet... .... eee', 'Consolidated profit and loss account ................ee', 'Notes the consolidated financial statements... neee', 'Balance sheet ............ eee', 'Profit and loss account... 1... 0.0... eee eee nes', 'Notes the financial statements... eee', 'Specifications the financial statements ......... 0.0.0.0... ccc']
---------
['Page', 'Momo Medical Holding B.V., DELFT', 'Consolidated balance sheet december', 'After appropriation results', '', 'EUR EUR', 'Assets', 'Non0current assets', 'Property, plant and equipment', 'Machinery 292 13,719', 'Other tangible assets 784,641 214,361', 'Total property, plant and equipment 828,933 228,080', 'Total non0current assets 828,933 228,080', 'Current assets', 'Inventories', 'Raw materials and consumables 266,885 63,896', 'Inventories under constru

In [14]:
# creating output from cleaned text

def try_parse_float(s):
    try:
        float(s.replace(',', '').replace('(', '').replace(')', ''))
        return True
    except ValueError:
        return False

def extract_lines_with_descriptions_and_numbers(document: Document):
    extracted_lines = []
    buffer_line = ""

    for page in document.pages:
        for line in page.lines:
            words = line.split()
            numeric_words = [word for word in words if try_parse_float(word)]
            non_numeric_words = [word for word in words if not try_parse_float(word)]

            # Check if the line has both text and numbers
            if non_numeric_words and numeric_words:
                # Add any buffered line before processing this line
                if buffer_line:
                    extracted_lines.append(buffer_line)
                    buffer_line = ""
                extracted_lines.append(line)
            # If the line contains only numbers, append to the buffer
            elif numeric_words and not non_numeric_words:
                if buffer_line:
                    buffer_line += " " + line
                else:
                    buffer_line = line
            # Reset buffer if the line contains only text
            elif non_numeric_words:
                if buffer_line:
                    buffer_line = ""
                buffer_line = ""

    # Add any remaining buffer line that contains numbers and text
    if buffer_line and any(try_parse_float(word) for word in buffer_line.split()) and any(not try_parse_float(word) for word in buffer_line.split()):
        extracted_lines.append(buffer_line)

    return extracted_lines

extracted_lines = extract_lines_with_descriptions_and_numbers(doc)
for line in extracted_lines:
    print(line)

Machinery 292 13,719
Other tangible assets 784,641 214,361
Total property, plant and equipment 828,933 228,080
Total non0current assets 828,933 228,080
Raw materials and consumables 266,885 63,896
Inventories under construction 42,450 25,064
Finished and trade goods 125,005 61,284
Total inventories 434,340 150,244
Trade receivables 182,558 229,329
Taxes and social security charges 8,647 7,149
Prepayments and accrued income 9,046 505,757
Total receivables 200,301 742,285
Cash and cash equivalents 219,434 44,377
Total current assets 854,075 936,906
Net revenue 598,589 198,331
Other operating income 61,100
Total operating income 598,589 259,431
Wages 778,978 316,901
Severance payments 5,340
Social security contributions 166,291 87,744
Other employee benefits expenses 170,964 86,948
Total expenses employee benefits 1,121,573 491,593
Depreciation machinery 7,856 4,312
Depreciation other tangible assets 77,081 35,727
Total depreciation property, plant and equipment 84,937 40,039
Total amorti

In [15]:
# extracting tables from the text

def try_parse_float(s):
    try:
        float(s.replace(',', '').replace('(', '').replace(')', ''))
        return True
    except ValueError:
        return False

def extract_lines_with_descriptions_and_numbers(document: Document):
    extracted_lines = []
    for page in document.pages:
        for line in page.lines:
            words = line.split()
            if len(words) > 1 and all(try_parse_float(word) for word in words[-2:]):
                extracted_lines.append(line)
    return extracted_lines

extracted_lines = extract_lines_with_descriptions_and_numbers(doc)
for line in extracted_lines:
    print(line)

Machinery 292 13,719
Other tangible assets 784,641 214,361
Total property, plant and equipment 828,933 228,080
Total non0current assets 828,933 228,080
Raw materials and consumables 266,885 63,896
Inventories under construction 42,450 25,064
Finished and trade goods 125,005 61,284
Total inventories 434,340 150,244
Trade receivables 182,558 229,329
Taxes and social security charges 8,647 7,149
Prepayments and accrued income 9,046 505,757
Total receivables 200,301 742,285
Cash and cash equivalents 219,434 44,377
Total current assets 854,075 936,906
Net revenue 598,589 198,331
Total operating income 598,589 259,431
Wages 778,978 316,901
Social security contributions 166,291 87,744
Other employee benefits expenses 170,964 86,948
Total expenses employee benefits 1,121,573 491,593
Depreciation machinery 7,856 4,312
Depreciation other tangible assets 77,081 35,727
Total depreciation property, plant and equipment 84,937 40,039
Total amortisation intangible assets and depreciation 84,937 40,039

In [16]:
# saving output in a table
def save_extracted_lines_to_table(extracted_lines):
    data = []
    for line in extracted_lines:
        words = line.split()
        numeric_words = [word for word in words if try_parse_float(word)]
        non_numeric_words = [word for word in words if not try_parse_float(word)]
        description = ' '.join(non_numeric_words)
        row = [description] + numeric_words
        data.append(row)

    # Determine the maximum number of columns needed
    max_columns = max(len(row) for row in data)

    # Pad rows with '-' to ensure all rows have the same number of columns
    for row in data:
        row.extend('-' * (max_columns - len(row)))

    # Create DataFrame
    columns = ['Description'] + [f'Year {i}' for i in range(1, max_columns)]
    df = pd.DataFrame(data, columns=columns)
    return df

df = save_extracted_lines_to_table(extracted_lines)
df_parsed = df

print(df)


NameError: name 'pd' is not defined

#### testing to delete

In [ ]:
class Table:
    rows = {}
    def __init__(self, page: Page):
        pass
        
    def parse_table(self, page: Page):
        # we need to determine which table type we are dealing with
        # first
        pass
        

    def __str__(self):
        return f"Header: {self.header}\nRows: {self.rows}"

In [ ]:
# Text just for testing
sample_text = """

Projectopbrengsten
Brutomarge

Kosten van grond en hulpstoffen
Kosten van uitbesteed werk

Lonen en Salarissen
Sociale lasten

Overige personeelskosten
Afschrijvingen

Overige bedrijfskosten
Totaal bedrijfskosten

Bedrijfsresuitaat

Andere rentebaten en soortgelijke opbrengsten

Rentelasten en soortgelijke kosten
Resultaat voor belastingen

Vennootschapsbelasting

Resultaat na belastingen

11.121.880
11.121.880

5.685.632
298.405
2.713.213
694.589
292.192
67.466
1.096.123
10.847.620

274.260

0

-187,548
86.712

2.963

89.675
"""
#TODO add condition for more than 1 numeric value in a line

# Split the text into lines and remove empty lines
lines = [line.strip() for line in sample_text.split('\n') if line.strip()]

# Separate descriptions and numbers
descriptions = []
numbers = []
for line in lines:
    if re.match(r'^[\d,.\-]+$', line):
        numbers.append(line.replace(',', ''))
    else:
        descriptions.append(line)

# Determine the minimum length to avoid index errors
min_length = min(len(descriptions), len(numbers))

# Slice descriptions list to take from the bottom if there are more descriptions
if len(descriptions) > len(numbers):
    descriptions = descriptions[-min_length:]

# Print descriptions and corresponding numbers in columns
for i in range(min_length):
    print(f"{descriptions[i]:40} {numbers[i]:>15}")

# Print a warning if there is a mismatch between descriptions and numbers
if len(descriptions) != len(numbers):
    print("\nWarning: Mismatch between the number of descriptions and numbers.")
    print(f"Descriptions count: {len(descriptions)}")
    print(f"Numbers count: {len(numbers)}")


In [ ]:
# another table type

def process_table_lines(text_data):
    lines = text_data.strip().split('\n')
    table_lines = []

    for line in lines:
        words = line.split()
        description = []
        values = []

        for word in words:
            if word.replace('.', '', 1).replace('-', '', 1).isdigit() or word == '_':
                values.append(word)
            else:
                description.append(word)
        
        # Skip line if there are no numeric values next to the text
        if len(values) == 0:
            continue
        
        # skip the line if there is no text before the numbers (it's totals from the subcategories)
        if len(description) == 0:
            continue
        
        description_text = ' '.join(description)
        table_lines.append((description_text, values))

    return table_lines

def format_table(table_lines):
    formatted_table = []

    # Header
    formatted_table.append("Description\tYear_1_Value\tYear_2_Value")

    # Body
    for description, values in table_lines:
        num_values = len(values)
        if num_values == 0:
            formatted_table.append(f"{description}\t\t")
        elif num_values == 1:
            formatted_table.append(f"{description}\t{values[0]}\t")
        elif num_values == 2:
            formatted_table.append(f"{description}\t{values[0]}\t{values[1]}")
        else:
            formatted_table.append(f"{description}\t{values[0]}\t{values[1]}\t{' '.join(values[2:])}")

    return formatted_table


cleaned_data = """
Hoofdsom 154.408 154.408
Cumulatieve aflossing -60.048 -46.704
Saldo per 01-01-2021 94.360 107.704
Aflossing 13344 _ 23344
Stand per 31-12-2021
Hoofdsom 154.408 154.408
Cumulatieve aflossing -73.392 -60.048
Kortlopend deel -13.344 -13.344
Stand per 31-12-2021 67,672 81.016
Kortlopende schulden 31-12-2021 31-12-2020
"""

table_lines = process_table_lines(cleaned_data)
formatted_table = format_table(table_lines)
for line in formatted_table:
    print(line)

## extracting text (descriptions) from pages

In [ ]:
# when using PyMuPDF

def extract_descriptions_pymupdf(pdf_path):
    doc = fitz.open(pdf_path)  # Open the PDF file
    pages_text = []  # Initialize a list to store text for each page

    for page in doc:  # Iterate through each page
        page_text = page.get_text()  # Extract text from the page
        filtered_text = ""  # Initialize a string to store filtered text

        # Split the extracted text into lines and filter out lines containing numeric values
        lines = page_text.split('\n')
        for line in lines:
            if not re.search(r'\d', line):  # Check if the line contains digits
                filtered_text += line + '\n'  # Add the line to filtered text if no digits found

        pages_text.append(filtered_text)  # Append the filtered text of the current page to the list

    doc.close()  # Close the document
    return pages_text  # Return a list of filtered text for each page


# Example usage
pages_text = extract_descriptions_pymupdf(pdf_path)  # Call the function to extract and filter text
for page_number, text in enumerate(pages_text, start=1):
    print(f"Page {page_number}:\n{text}\n---\n")  # Print each page's text

In [17]:
## for OCR 

page_number = 1  # Initialize page number counter

for page in text_description.pages:
    filtered_lines = []  # Prepare a list to hold filtered lines

    # Iterate through each line in the current page
    for line in page.lines:
        # Check if the line contains any digits
        if not re.search(r'\d', line):
            filtered_lines.append(line)

    # Join the filtered lines into a single string for output
    filtered_text = '\n'.join(filtered_lines)

    # Output the page number and the filtered text
    print(f"Page {page_number}:\n{filtered_text}\n---\n")
    
    # Increment the page number for the next iteration
    page_number += 1

Page 1:
The accountant's compilation report has been issued
---

Page 2:
Consolidated financial statements
Momo Medical Holding B.V.
DELFT
---

Page 3:
Table contents
Consolidated balance sheet... .... eee
Consolidated profit and loss account ................ee
Notes the consolidated financial statements... neee
Balance sheet ............ eee
Notes the financial statements... eee
---

Page 4:
Page
Momo Medical Holding B.V., DELFT
Consolidated balance sheet december
After appropriation results

EUR EUR
Assets
Property, plant and equipment
Current assets
Inventories
Receivables
Current receivables from other legal entities and companies with
participating interest the legal entity from participating interests
the legal entity
Total assets
---

Page 5:
Liabilities
Group equity
Debentures, mortgage bonds and other loans
Payables banks
Payables relating taxes and social security contributions
Current liabilities
Advances received
Trade payables
Payables relating taxes and social security co

## getting company info

In [ ]:
### getting company info from file name

#get the file name from path
file_name = pdf_path.split("/")[-1]
#transform the file name to lower case
file_name = file_name.lower()
#remove the words pdf, Jaarverslag and Jaarrekening,... from the file name
file_name = file_name.replace(".pdf", "").replace("jaarverslag", "").replace("jaarrekening", "").replace("geconsolideerd", "").replace("geconsolideerde", "").replace("annual", "").replace("report", "").replace("consolidated", "").replace("financial", "").replace("statement", "").replace("statements", "") .replace("jaarbericht", "") 

# if there is a number in the file name, it is the year
year = re.findall(r'\d{4}', file_name)
# company name is the file name without the year
company_name = file_name.replace(year[0], "").replace("-", " ").strip()

#from the first 1000 characters of the text, extract the currency
#TODO: add more currencies, improve logic
currency = re.findall(r'€|usd|dollar', text[:1000].lower())
if currency:
    currency = currency[0]
else:
    currency = "EUR"

#year is Year_1, Year_2 is the year -1, display the extracted company name and year 1 and 
company_information_df = pd.DataFrame([[company_name, year[0], str(int(year[0])-1), currency ]], columns=['Company Name', 'Year 1', 'Year 2', 'Currency'])


display(company_information_df)


## 2. Extracting, Categorising, and Processing data 

In [ ]:
# TODO: different way to extract information -> from file names, this only works correctly for English documents, new approach in cell below
# extracting company information
client = OpenAI(base_url = 'http://localhost:11434/v1',api_key='ollama')

def prepare_prompt(text: str) -> list:
    return [
        # prompt not final, to be adjusted
        {"role": "system", "content": " extract the name of the company, the two years that the financial statement is about (i.e 2021, 2022), in the order they appear, the currency, and the type of financial statement. Output 5 (not more) variables, separated by commas."},
        {"role": "user", "content": text},
        {"role": "assistant", "content": "category name:"}
    ]

def output_company_information(text: str) -> str:
    messages = prepare_prompt(text)
    response = client.chat.completions.create(
        # TODO: try on smaller models
        model="gemma:7b",
        messages = messages,
        temperature=0.4
    )
  
    company_information = response.choices[0].message.content
    return company_information

# from the first 500 characters of df_parsed, extract company name, year, currency, and the type of financial statement
company_information = output_company_information(text[:500])
#make company informarion only the first row of the output
company_information = company_information.split("\n")[0]
print(company_information)


# make a new dataframe with the extracted company information
company_information = company_information.split(",")
company_information_df = pd.DataFrame([company_information], columns=['Company Name', 'Year 1', 'Year 2', 'Currency', 'Type of Financial Statement'])
display(company_information_df)

In [ ]:
# setup for replacing all outputs longer than XXX characters with a category name extracted by gemma

def prepare_messages(text: str) -> list:
    return [
        # prompt not final, to be adjusted
        {"role": "system", "content": "This is a fragment of description from a financial statement. Extract a financial category name from the description. Output only the category and no other text or explanations. If there is no category or not applicable, output only: -."},
        {"role": "user", "content": text},
        {"role": "assistant", "content": "category name:"}
    ]

def output_category_name(text: str) -> str:
    messages = prepare_messages(text)
    response = client.chat.completions.create(
        # TODO: try on smaller models
        model="gemma:7b",
        messages = messages,
        temperature=0.8
    )
    category_name = response.choices[0].message.content
     # convert to lowercase
    category_name = category_name.lower()
    # remove whitespaces
    category_name = category_name.strip()
    return category_name


In [ ]:
MAX_DESCRIPTION_LENGTH = 220
LLM_length=100

#change the 'Year 1 EUR' and 'Year 2 EUR' columns in df_parsed to the values under Year 1 and Year 2 in company_information_df
client = OpenAI(base_url = 'http://localhost:11434/v1',api_key='ollama')

df_parsed = df_parsed.rename(columns={'Year 1': company_information_df['Year 1'][0], 'Year 2': company_information_df['Year 2'][0]})
df_parsed['Category'] = df_parsed['Description']

# extracring category names for descriptions longer than LLM_length
for i, row in enumerate(df_parsed['Description']):
    if len(row) > LLM_length and len(row) < MAX_DESCRIPTION_LENGTH:
        category_name = output_category_name(row)
        # tentative solution, to be deleted once the prompt is adjusted
        category_name = category_name.split("\n")[0]
        df_parsed.at[i, 'Category'] = category_name
        print(row, "->", category_name)
  


### Mapping data to ledger

In [ ]:
from financial_ledger import financial_ledger as ledger
import jellyfish as jf
from jellyfish import jaro_winkler_similarity as jws
from tqdm import tqdm

In [ ]:
# functions for calculating the Jaro-Winkler similarity for word matching
# a string metric measuring an edit distance between two sequences.
# edit distance is measured by counting the minimum number of operations required to transform one string into the othe

def add_matching_info_to_df(df, ledger):
    # Initialize lists to hold match results
    best_matches = []
    match_scores = []
    
    for index, row in df.iterrows():
        description = row['Description'].lower()
        max_score = 0
        best_match = ""
        for key, ledger_entry in ledger.items():
            # Iterate through both 'English' and 'Dutch' lists if they exist
            for lang in ['English', 'Dutch']:
                if lang in ledger_entry:  # Check if the language key exists
                    for term in ledger_entry[lang]:
                        score = jf.jaro_winkler(description, term.lower())
                        if score > max_score:
                            max_score = score
                            best_match = key
        # Append match result or indicate no match found
        if max_score > 0.7:
            best_matches.append(best_match)
            match_scores.append(max_score)
        else:
            best_matches.append("No match found")
            match_scores.append(max_score)
    
    # Add the match results to the DataFrame
    df['Best Match_JW'] = best_matches
    df['Match Score_JW'] = match_scores

add_matching_info_to_df(df_parsed, ledger)

# Now df_parsed contains two new columns: 'Best Match' and 'Match Score'
display(df_parsed)

#display only the rows with no match found
# df_no_match = df_parsed[df_parsed['Best Match_JW'] == "No match found"]
# display(df_no_match)


In [ ]:
# NOTE: do not run unless you want to display all rows
# this is used to display all rows in the dataframe when checking output, not necessary to run as it makes the output very long, only for testing/checking purposes

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Auto-detect the display width
pd.set_option('display.max_colwidth', -1)  # Display full width of columns

#display(df_parsed)

In [ ]:
# cosine similarity

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np


# Combine all texts to build the vocabulary for vectorization
all_texts = list(df_parsed['Category']) + [item for sublist in ledger.values() for lang in sublist if lang in ['English', 'Dutch'] for item in sublist[lang]]

vectorizer = TfidfVectorizer().fit(all_texts)
description_vectors = vectorizer.transform(df_parsed['Category'])

# Prepare and vectorize ledger entries
ledger_entries = [item for sublist in ledger.values() for lang in sublist if lang in ['English', 'Dutch'] for item in sublist[lang]]
ledger_vectors = vectorizer.transform(ledger_entries)

# Calculate cosine similarity between description vectors and ledger vectors
similarity_scores = cosine_similarity(description_vectors, ledger_vectors)

# Determine the best match for each description based on the highest cosine similarity score
best_matches = [ledger_entries[np.argmax(row)] if max(row) > 0.7 else "No match found" for row in similarity_scores]
df_parsed['Best Match_Cosine'] = best_matches
df_parsed['Highest Match Score_Cosine'] = [max(row) for row in similarity_scores]

# for each value in `Best Match_Cosine` assign the corresponding higher level key from the ledger
for i, row in enumerate(df_parsed['Best Match_Cosine']):
    for key, value in ledger.items():
        if row in value['English'] or row in value['Dutch']:
            df_parsed.at[i, 'Best Match_Cosine'] = key
            break

display(df_parsed)


## 3. Creating output

In [ ]:
# if in df_parsed, best match JR and best match cosine  = "not found" or "not applicable", delete the row
# TODO: check if when there is no match from one method, if there is one from the other 

#if there is no match in 'Best Match_Cosine', replace it with the match from 'Best Match_JW'
df_parsed['Best Match_Cosine'] = np.where(df_parsed['Best Match_Cosine'] == "No match found", df_parsed['Best Match_JW'], df_parsed['Best Match_Cosine'])

df_parsed = df_parsed[df_parsed['Best Match_Cosine'] != "No match found"]
df_parsed = df_parsed[df_parsed['Best Match_Cosine'] != "not applicable"]

# if categories are found, make a new df with catgegory name, year 1 value,  year 2 value, output info from the ledger like id and other categories based on best match
df_output = df_parsed[['Category', company_information_df['Year 1'][0], company_information_df['Year 2'][0], 'Best Match_JW', 'Match Score_JW', 'Best Match_Cosine', 'Highest Match Score_Cosine']]

# based on the best match in the ledger, make new columns with balance, id, category, and statement type from the ledger for each row in df_output
def add_ledger_info_to_df(df, ledger):
    # Initialize lists to hold match results
    balance = []
    id = []
    category = []
    statement_type = []
    postencode= []
     
    for index, row in df.iterrows():
        best_match = row['Best Match_Cosine']
        for key, ledger_entry in ledger.items():
            if key == best_match:
                balance.append(ledger_entry['balance'])
                id.append(ledger_entry['id'])
                category.append(ledger_entry['category'])
                statement_type.append(ledger_entry['statement_type'])
                postencode.append(ledger_entry['postencode'])
    
    # Add the match results to the DataFrame
    df_output['Balance'] = balance
    df_output['ID'] = id
    df_output['Category'] = category
    df_output['Statement Type'] = statement_type
    df_output['Postencode'] = postencode

    
add_ledger_info_to_df(df_output, ledger)
display(df_output)
df_output['Postencode'] = df_output['Postencode'].astype('int64')



In [ ]:
# import excel file from path: /Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/20231212_kvk_finposten_dictionary - Antwoorden.xlsx
kvk_file_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/20231212_kvk_finposten_dictionary - Antwoorden.xlsx'
kvk_df = pd.read_excel(kvk_file_path, sheet_name='Postencodes')
# leave only the first 5 columns
kvk_df = kvk_df.iloc[:, :5]

#add a row with postencode equal to 1 and all other columns equal to "totals"
kvk_df.loc[-1] = [1, 'Total', 'Total', 'Total', 'Total']
kvk_df.index = kvk_df.index + 1
kvk_df = kvk_df.sort_index()
#display(kvk_df)


In [ ]:
#join output df with kvk_df on the postencode column
df_output = df_output.merge(kvk_df, how='left', left_on='Postencode', right_on='Postencode')

display(df_output)

In [ ]:
#delete the Match Score_JW	Best Match_Cosine, Highest Match Score_Cosine, Category, Balance, and ID columns 
df_output = df_output.drop(columns=['Match Score_JW', 'Best Match_JW', 'Highest Match Score_Cosine', 'Balance', 'ID', "Statement Type"])
# display NaN as '-'
df_output = df_output.fillna('-')

# display the dataframe
display(df_output)


In [ ]:
print(df_output.columns)

# Melting the DataFrame to have 'year' and 'value' columns

df_melted = df_output.melt(id_vars=["Category", "Postencode", "Hoofd posten code:", "Balans/res.rek. indicatie:", "Debet/credit indicatie: ", "Omschr. postencode: "],
                           value_vars=["2022", "2021"],
                           var_name="Year", value_name="Value")

    
# add company name and currency to the df
df_melted['Company Name'] = company_information_df['Company Name'][0]
df_melted['Currency'] = company_information_df['Currency'][0]

# delete the category column
# df_melted = df_melted.drop(columns=['Category'])

display(df_melted)

# save the dataframe to a csv file
df_melted.to_csv('output.csv', index=False)




## 4. Data Validation


In [ ]:
# from Category of df_melted, check whether assets= liabilities + equity

df_checks = df_melted[df_melted['Postencode'] != 1]    

# from Category of df_melted, sum all values for assets, liabilities, and equity for each year
assets_y1 = df_checks[(df_checks['Category'] == 'assets') & (df_checks['Year'] == company_information_df['Year 1'][0])]['Value'].sum()
liabilities_y1 = df_checks[(df_checks['Category'] == 'liabilities') & (df_checks['Year'] == company_information_df['Year 1'][0])]['Value'].sum()
equity_y1 = df_checks[(df_checks['Category'] == 'equity') & (df_checks['Year'] == company_information_df['Year 1'][0])]['Value'].sum()

# check if the sum of assets is equal to the sum of liabilities and equity
if assets_y1 == liabilities_y1 + equity_y1:
    print("The balance sheet balances for", company_information_df['Year 1'][0])
else:
    print("The balance sheet does not balance for", company_information_df['Year 1'][0])

# print the values for assets, liabilities, and equity for year 1
print("Assets:", assets_y1)
print("Liabilities:", liabilities_y1)
print("Equity:", equity_y1)

#print sum of equity and liabilities
print("Sum of liabilities and equity:", liabilities_y1 + equity_y1)


#perform the same check for year 2
assets_y2 = df_checks[(df_checks['Category'] == 'assets') & (df_checks['Year'] == company_information_df['Year 2'][0])]['Value'].sum()
liabilities_y2 = df_checks[(df_checks['Category'] == 'liabilities') & (df_checks['Year'] == company_information_df['Year 2'][0])]['Value'].sum()
equity_y2 = df_checks[(df_checks['Category'] == 'equity') & (df_checks['Year'] == company_information_df['Year 2'][0])]['Value'].sum()

# check if the sum of assets is equal to the sum of liabilities and equity
if assets_y2 == liabilities_y2 + equity_y2:
    print("The balance sheet balances for", company_information_df['Year 2'][0])
else:
    print("The balance sheet does not balance for", company_information_df['Year 2'][0])
            
    